In [1]:
import pickle
import json

In [2]:
import scipy.spatial.distance as dist

In [3]:
import cv2
import argparse
import numpy as np

In [14]:
path='state_test1.jpeg'

In [5]:
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

In [6]:
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [16]:
def object_detector(path,classes):
    image = cv2.imread(path)
    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392
    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
    WEIGHT=r'yolov3.weights'
    CONFIG=r'yolov3.cfg'
    net = cv2.dnn.readNet(WEIGHT,CONFIG )
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(get_output_layers(net))
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    return boxes,class_ids

In [25]:
class object_type():
    def __init__(self,centre,landmark_coordinates,classid):
        self.classid=classid
        self.centre=centre
        self.landmark_coordinates=landmark_coordinates
        self.distance=self.echo_locator(self.centre,self.landmark_coordinates)
        self.state=self.state_determiner(self.centre,self.landmark_coordinates)
    def echo_locator(self,centre,landmark_coordinates):
        distance=0
        for i in range(len(landmark_coordinates)):
            distance += dist.euclidean(centre,landmark_coordinates[i])
        distance=distance/i
        return distance 
    def state_determiner(self,centre,landmark_coordinates):
        distance_close=0
        state=0
        for i in range(len(landmark_coordinates)):
            distance_temp=dist.euclidean(centre,landmark_coordinates[i])
            if(distance_close>distance_temp):
                distance_close=distance_temp
                state=i
            else:
                continue
        return state   

In [29]:
def object_initiator(path,class_type,landmark_coordinates):
    classes = None
    CLASSES_FILE=r'yolov3.txt'
    with open(CLASSES_FILE, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    object_dict={}
    boxes,class_ids=object_detector(path,classes)
    for i in range(len(boxes)):
        index=int(class_ids[i])
        if(classes[index]=='bed' or classes[index]=='chair'):
            continue
        else:
            temp=classes[class_ids[i]]
            object_dict[temp]=class_type((boxes[i][0],boxes[i][1]),landmark_coordinates,index)
    return object_dict,classes           

In [26]:
landmark=open('landmark','rb')
landmark_coordinates=pickle.load(landmark)
landmark.close()

In [38]:
#THE MAIN FUNCTION
def feat_vector_updater(path,class_type,landmark_coordinates):
    object_dict,classes=object_initiator(path,object_type,landmark_coordinates)
    flag=0
    for key in object_dict:
        if(key=='person'):
            feat_vector=[]
            feat_vector.append(object_dict[key].state)
            feat_vector.append(object_dict[key].distance)
            feat_vector.append(object_dict[key].classid)
            flag=1
        else:
            continue       
    if(flag==0):
        feat_vector=[]
        feat_vector.append(42)
        for key in object_dict:
            feat_vector.append(object_dict[key].state)
            feat_vector.append(object_dict[key].distance)
            feat_vector.append(object_dict[key].classid)
    else:
        for key in object_dict:
            if(key=='person'):
                continue
            else:
                feat_vector.append(object_dict[key].state)
                feat_vector.append(object_dict[key].distance)
                feat_vector.append(object_dict[key].classid)
    with open('object_feat_vector','wb') as f:
        pickle.dump(feat_vector,f)
    return 0